In [1]:
#!pip install unidecode
#!pip install nltk

In [2]:
import collections
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
import nltk
import os
import pandas as pd
import pathlib
import plotly.express as px
import re
from unidecode import unidecode

In [3]:
os.getcwd()

'/home/jovyan/work/tweets'

In [4]:
path = "/home/jovyan/work/tweets/data/"
os.chdir(path)

# read data

In [5]:
file_ext = "."
files = os.listdir(file_ext)

files_tw = []
for filename in files:
    try :
        tmp = re.search(r"tw_.*\b", filename)[0]
        if tmp:
            files_tw.append(tmp)
    except:
        pass 


data = pd.DataFrame()
for file in files_tw:
    _tx = pd.read_csv(f"{path}{file}", sep=" ")
    _tx.columns = ['datetime', 'tweet_id', 'text',
                  'username', 'like_count', 'retweet_count',
                  'reply_count', 'quote_count']
    print(_tx.shape)
    data = pd.concat([data, _tx], axis=0)
data = data.reset_index(drop=True)
    

(4399, 8)
(5014, 8)


In [6]:
# Check index duplicates 

In [7]:
data[data.duplicated(keep=False)]

,datetime,tweet_id,text,username,like_count,retweet_count,reply_count,quote_count


In [8]:
data.head()

,datetime,tweet_id,text,username,like_count,retweet_count,reply_count,quote_count
0,2022-02-01 06:27:17+00:00,1488398530075148292,@BFMTV nous survenu une hypothétique guerre en...,fbain_,0,0,0,0
1,2022-02-01 06:27:14+00:00,1488398518423277570,Ce philosophe est un va-t-en guerre !\nEst-ce ...,karax12,0,0,0,0
2,2022-02-01 06:26:11+00:00,1488398252017954818,Le ministre #turc de la Défense évoque l'impor...,nryunus1,4,0,0,0
3,2022-02-01 06:25:28+00:00,1488398072791150594,J'ai compris le problème du Président @Emmanue...,datadito,0,0,0,0
4,2022-02-01 06:24:27+00:00,1488397818347671556,#FLASHINFO\n#Ukraine: les menaces de sanctions...,ArabNewsfr,0,0,0,0


# Analyse exploratoire ==> julie

In [9]:
def transform_dates(df, column="datetime"):
    df[column] = df[column].apply(lambda x: pd.Timestamp(x))
    df["date"] = df[column].dt.date
    df["day"] = df[column].dt.day
    df["month"] = df[column].dt.month
    df["time"] = df[column].dt.time
    df["hour"] = df[column].dt.hour
    df["minute"] = df[column].dt.minute
    return df

In [10]:
data = transform_dates(df=data, column="datetime")

In [11]:
data["datetime"].min()

Timestamp('2022-02-01 06:02:44+0000', tz='UTC')

In [12]:
data["datetime"].max()

Timestamp('2022-02-21 10:28:17+0000', tz='UTC')

tmp = data.groupby("day")["tweet_id"].count().reset_index()
display(tmp)
fig = px.line(tmp, x='day', y="tweet_id")
fig.show()

# Monitorer l'évolution du #ukraine ==> Alicia

# Preprocessing ==> Anne + Valentina

### - Filter only french tweets  @julie

In [13]:
# @julie

In [14]:
data["text"]

0       @BFMTV nous survenu une hypothétique guerre en...
1       Ce philosophe est un va-t-en guerre !\nEst-ce ...
2       Le ministre #turc de la Défense évoque l'impor...
3       J'ai compris le problème du Président @Emmanue...
4       #FLASHINFO\n#Ukraine: les menaces de sanctions...
                              ...                        
9408    Crise en #Ukraine : La Russie a l’intention de...
9409    #Ukraine: L'Élysée confirme un sommet entre Po...
9410    #ukraine , c'est une blague...😳😳😳👇👇https://t.c...
9411    @Beckerjean @sandrousseau Les Ukrainiens le VE...
9412    L’agence de presse russe TASS annonce qu’un po...
Name: text, Length: 9413, dtype: object

### save "#" and "@" on a column

In [16]:
#test["top_hashtag"] = test["text"].apply(lambda x: re.match(str(x),"[#].*" ))
#Find all "#"
data["hashtags"] = data["text"].apply(lambda y: [x.group() for x in re.finditer(r'#[a-zA-Z]+', y)])

# Find all @
data["arrobas"] = data["text"].apply(lambda y: [x.group() for x in re.finditer(r'@[a-zA-Z]+', y)])

# lists of all hashtags
all_hashtags = [i for htg in data["hashtags"] for i in htg]
list_unique_htg = list(set(all_hashtags))
print(len(list_unique_htg))

2938


In [17]:
# lower case tweets
data["text"] = data["text"].str.lower()

# replace https links with "link"
regex_http = r'([\w+]+\:\/\/)?([\w\d-]+\.)*[\w-]+[\.\:]\w+([\/\?\=\&\#\.]?[\w-]+)*\/?'
data["text_clean"] = [re.sub(regex_http, '<link>', doc) for doc in data["text"]]

#data["text_clean"] = [re.sub(r'#[a-zA-Z]+', '<hashtag>', doc) for doc in data["text_clean"]]
#data["text_clean"] = [re.sub(r'@[a-zA-Z]+', '<arrobas>', doc) for doc in data["text_clean"]]

# replace years
data["text_clean"] = [re.sub(r'[0-9]{4}', '<annee>', doc) for doc in data["text_clean"]]

In [18]:
data["text_clean"]

0       @bfmtv nous survenu une hypothétique guerre en...
1       ce philosophe est un va-t-en guerre !\nest-ce ...
2       le ministre #turc de la défense évoque l'impor...
3       j'ai compris le problème du président @emmanue...
4       #flashinfo\n#ukraine: les menaces de sanctions...
                              ...                        
9408    crise en #ukraine : la russie a l’intention de...
9409    #ukraine: l'élysée confirme un sommet entre po...
9410            #ukraine , c'est une blague...😳😳😳👇👇<link>
9411    @beckerjean @sandrousseau les ukrainiens le ve...
9412    l’agence de presse russe tass annonce qu’un po...
Name: text_clean, Length: 9413, dtype: object

In [19]:
# delete accents and emojis
data["text_clean"] = [unidecode(doc) for doc in data["text_clean"]]

In [20]:
# replace all non alphabetic chars with an space
data["text_clean"] = [re.sub(r"[^a-z<>]+", ' ', doc) for doc in data["text_clean"]]

In [ ]:
# exemple

In [25]:
data["text"].iloc[4]

'#flashinfo\n#ukraine: les menaces de sanctions ne feront pas "battre en retraite" #moscou (ambassade) https://t.co/hxs8y92iiw'

In [26]:
data["text_clean"].iloc[4]

'flashinfo ukraine menaces sanctions feront battre retraite moscou ambassade <link>'

In [23]:
# French stopwords
nltk.download('stopwords')
french_stopwords_list = stopwords.words('french')
data["text_clean"] = [' '.join([word for word in doc.split() if word not in french_stopwords_list]) for doc in data["text_clean"]]

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# idees : supprimer les mots d'1 seule syllabe

# Nuage de mots ==> Anne